## **PART-2**


pip install torch torchvision torchaudio

In [2]:
import torch, torchvision
transform = torchvision.transforms.Compose( [torchvision.transforms.ToTensor(),
torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
  img = img / 2 + 0.5
  npimg = img.numpy()
  plt.figure(figsize = (3,3))
  plt.imshow(np.transpose(npimg, (1, 2, 0)))


In [4]:
total_size = len(trainset)
train_size = int(0.9 * total_size)
val_size = total_size - train_size

In [5]:
train_dataset, val_dataset = torch.utils.data.random_split(trainset, [train_size, val_size])

In [6]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

In [7]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, img_input):
        img_input = self.conv1(img_input)
        img_input = self.relu1(img_input)
        img_input = self.pool1(img_input)
        img_input = self.conv2(img_input)
        img_input = self.relu2(img_input)
        img_input = self.pool2(img_input)
        img_input = img_input.view(img_input.size(0), -1)  # Flatten
        img_input = self.fc1(img_input)
        img_input = self.relu3(img_input)
        img_input = self.fc2(img_input)
        return img_input

cnn_model = SimpleCNN()
criterion_lf = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn_model.parameters(), lr=0.01, momentum=0.9)

def train(cnn_model, train_loader, criterion_lf, optimizer, processor):
    cnn_model.train()
    total_loss = 0.0
    correct_op = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(processor), labels.to(processor)
        optimizer.zero_grad()
        outputs = cnn_model(inputs)
        loss_cal = criterion_lf(outputs, labels)
        loss_cal.backward()
        optimizer.step()
        total_loss += loss_cal.item()
        _, predicted_val = outputs.max(1)
        total += labels.size(0)
        correct_op += predicted_val.eq(labels).sum().item()
    train_accuracy = 100.0 * correct_op / total
    return total_loss / len(train_loader), train_accuracy

def validate(cnn_model, val_loader,criterion_lf, processor):
    cnn_model.eval()
    total_loss = 0.0
    correct_op = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(processor), labels.to(processor)
            outputs = cnn_model(inputs)
            loss_cal = criterion_lf(outputs, labels)
            total_loss += loss_cal.item()
            _, predicted_val = outputs.max(1)
            total += labels.size(0)
            correct_op += predicted_val.eq(labels).sum().item()
    val_accuracy = 100.0 * correct_op / total
    return total_loss / len(val_loader), val_accuracy

def test(cnn_model, test_loader, processor):
    cnn_model.eval()
    correct_op = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(processor), labels.to(processor)
            outputs = cnn_model(inputs)
            _, predicted_val = outputs.max(1)
            total += labels.size(0)
            correct_op += predicted_val.eq(labels).sum().item()
    test_accuracy = 100.0 * correct_op / total
    return test_accuracy

processor = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_epochs = 10
for epoch in range(num_epochs):
    train_loss, train_accuracy = train(cnn_model.to('cuda'), train_loader, criterion_lf, optimizer, processor)
    val_loss, val_accuracy = validate(cnn_model.to('cuda'), val_loader, criterion_lf, processor)
    test_accuracy = test(cnn_model.to('cuda'), test_loader, processor)
    print(f"Epoch {epoch + 1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%")
    print(f"Validation Loss: {val_loss:.4f} | Validation Accuracy: {val_accuracy:.2f}%")
    print(f"Test Accuracy: {test_accuracy:.2f}%")


Epoch 1/10:
Train Loss: 1.6067 | Train Accuracy: 41.49%
Validation Loss: 1.2816 | Validation Accuracy: 53.64%
Test Accuracy: 53.62%
Epoch 2/10:
Train Loss: 1.1851 | Train Accuracy: 57.74%
Validation Loss: 1.2096 | Validation Accuracy: 56.90%
Test Accuracy: 57.01%
Epoch 3/10:
Train Loss: 1.0065 | Train Accuracy: 64.38%
Validation Loss: 1.0012 | Validation Accuracy: 64.20%
Test Accuracy: 64.67%
Epoch 4/10:
Train Loss: 0.8803 | Train Accuracy: 69.06%
Validation Loss: 0.9072 | Validation Accuracy: 67.94%
Test Accuracy: 68.15%
Epoch 5/10:
Train Loss: 0.7824 | Train Accuracy: 72.42%
Validation Loss: 0.9374 | Validation Accuracy: 67.28%
Test Accuracy: 67.24%
Epoch 6/10:
Train Loss: 0.6840 | Train Accuracy: 75.82%
Validation Loss: 0.9158 | Validation Accuracy: 68.48%
Test Accuracy: 68.08%
Epoch 7/10:
Train Loss: 0.5936 | Train Accuracy: 78.95%
Validation Loss: 0.9312 | Validation Accuracy: 69.50%
Test Accuracy: 69.07%
Epoch 8/10:
Train Loss: 0.5002 | Train Accuracy: 82.33%
Validation Loss: 0.9

In [8]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
total_size = len(trainset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size
train_dataset, val_dataset = torch.utils.data.random_split(trainset, [train_size, val_size])
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

class CNN5Layers(nn.Module):
    def __init__(self):
        super(CNN5Layers, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        pool1_output_size_calc = (32 - 2 + 2 * 1) // 2 + 1
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        pool2_output_size_calc = (pool1_output_size_calc - 2 + 2 * 1) // 2 + 1
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        pool3_output_size_calc = (pool2_output_size_calc - 2 + 2 * 1) // 2 + 1
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        pool4_output_size_calc = (pool3_output_size_calc - 2 + 2 * 1) // 2 + 1
        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu5 = nn.ReLU()
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(8192, 4096)
        self.relu6 = nn.ReLU()
        self.fc2 = nn.Linear(4096, 4096)
        self.relu7 = nn.ReLU()
        self.fc3 = nn.Linear(4096, 10)

    def forward(self, img_input):
        img_input = self.conv1(img_input)
        img_input = self.relu1(img_input)
        img_input = self.pool1(img_input)
        img_input = self.conv2(img_input)
        img_input = self.relu2(img_input)
        img_input = self.pool2(img_input)
        img_input = self.conv3(img_input)
        img_input = self.relu3(img_input)
        img_input = self.conv4(img_input)
        img_input = self.relu4(img_input)
        img_input = self.conv5(img_input)
        img_input = self.relu5(img_input)
        img_input = self.pool5(img_input)
        img_input = img_input.view(img_input.size(0), -1)  # Flatten
        img_input = self.fc1(img_input)
        img_input = self.relu6(img_input)
        img_input = self.fc2(img_input)
        img_input = self.relu7(img_input)
        img_input = self.fc3(img_input)
        return img_input

cnn_model = CNN5Layers()
criterion_lf = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn_model.parameters(), lr=0.01, momentum=0.9)
processor = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_epochs = 10
for epoch in range(num_epochs):
    train_loss, train_accuracy = train(cnn_model.to('cuda'), train_loader, criterion_lf, optimizer, processor)
    val_loss, val_accuracy = validate(cnn_model.to('cuda'), val_loader, criterion_lf, processor)
    test_accuracy = test(cnn_model.to('cuda'), test_loader, processor)

    print(f"Epoch {epoch + 1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%")
    print(f"Validation Loss: {val_loss:.4f} | Validation Accuracy: {val_accuracy:.2f}%")
    print(f"Test Accuracy: {test_accuracy:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/10:
Train Loss: 1.9999 | Train Accuracy: 24.95%
Validation Loss: 1.7391 | Validation Accuracy: 35.63%
Test Accuracy: 36.22%
Epoch 2/10:
Train Loss: 1.5383 | Train Accuracy: 43.11%
Validation Loss: 1.4079 | Validation Accuracy: 47.80%
Test Accuracy: 47.97%
Epoch 3/10:
Train Loss: 1.2755 | Train Accuracy: 53.30%
Validation Loss: 1.2161 | Validation Accuracy: 55.52%
Test Accuracy: 55.37%
Epoch 4/10:
Train Loss: 1.0723 | Train Accuracy: 61.52%
Validation Loss: 0.9677 | Validation Accuracy: 65.56%
Test Accuracy: 65.80%
Epoch 5/10:
Train Loss: 0.9237 | Train Accuracy: 67.04%
Validation Loss: 0.8958 | Validation Accuracy: 68.18%
Test Accuracy: 68.56%
Epoch 6/10:
Train Loss: 0.8135 | Train Accuracy: 71.41%
Validation Loss: 0.7872 | Validation Accuracy: 71.98%
Test Accuracy: 72.95%
Epoch 7/10:
Train Loss: 0.7377 | Train Accuracy: 74.12%
Validation Loss: 0.7511 | Validation Accuracy: 73.07%
Test Accuracy: 73.73%


In [9]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )

    def forward(self, img_input):
        img_input = self.conv_layers(img_input)
        img_input = self.fc_layers(img_input)
        return img_input

cnn_model = CustomCNN()
criterion_lf = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn_model.parameters(), lr=0.01, momentum=0.9)
processor = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_epochs = 20
for epoch in range(num_epochs):
    train_loss, train_accuracy = train(cnn_model.to('cuda'), train_loader, criterion_lf, optimizer, processor)
    val_loss, val_accuracy = validate(cnn_model.to('cuda'), val_loader, criterion_lf, processor)
    test_accuracy = test(cnn_model.to('cuda'), test_loader, processor)
    print(f"Epoch {epoch + 1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%")
    print(f"Validation Loss: {val_loss:.4f} | Validation Accuracy: {val_accuracy:.2f}%")
    print(f"Test Accuracy: {test_accuracy:.2f}%")


Epoch 1/20:
Train Loss: 2.2693 | Train Accuracy: 12.34%
Validation Loss: 2.0439 | Validation Accuracy: 22.50%
Test Accuracy: 22.16%
Epoch 2/20:
Train Loss: 1.9462 | Train Accuracy: 24.25%
Validation Loss: 1.8077 | Validation Accuracy: 30.11%
Test Accuracy: 30.53%
Epoch 3/20:
Train Loss: 1.7005 | Train Accuracy: 34.61%
Validation Loss: 1.5240 | Validation Accuracy: 42.97%
Test Accuracy: 42.85%
Epoch 4/20:
Train Loss: 1.4803 | Train Accuracy: 45.16%
Validation Loss: 1.3355 | Validation Accuracy: 51.28%
Test Accuracy: 51.06%
Epoch 5/20:
Train Loss: 1.2831 | Train Accuracy: 53.71%
Validation Loss: 1.1684 | Validation Accuracy: 57.42%
Test Accuracy: 57.14%
Epoch 6/20:
Train Loss: 1.1378 | Train Accuracy: 59.56%
Validation Loss: 1.1299 | Validation Accuracy: 59.58%
Test Accuracy: 59.55%
Epoch 7/20:
Train Loss: 1.0092 | Train Accuracy: 64.25%
Validation Loss: 0.9650 | Validation Accuracy: 66.46%
Test Accuracy: 67.37%
Epoch 8/20:
Train Loss: 0.9099 | Train Accuracy: 68.31%
Validation Loss: 0.8

**Methods and Results:**
The first model, which consists of two CNN layers, achieved a testing accuracy of 68.11%.

In the second model, the number of layers was increased to five CNN layers, resulting in a significant improvement in accuracy by approximately 10-11%, with the second model achieving an accuracy of around 78.45%.

For the third model, six CNN layers were utilized, and a dropout layer was incorporated. This model achieved an even higher accuracy of 83.66%.